In [ ]:
import requests
import os
import time
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# "coin list" download

In [ ]:
def get_coin_list(isactive:bool):
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/map'

    headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': 'e0070978-f2c5-4a31-b9b8-7705d84077bd',
    }

    isActive = 'active' if isactive else 'inactive'
    params = {
        'listing_status': isActive,
    }

    res = requests.get(url=url, headers=headers, params=params)

    return res

In [ ]:
# active랑 inactive를 가져옴
active = get_coin_list(isactive=True)
inactive = get_coin_list(isactive=False)

In [ ]:
active_data = active.json()["data"]
inactive_data = inactive.json()["data"]

In [ ]:
df_active = pd.DataFrame(active_data)
df_inactive = pd.DataFrame(inactive_data)
df_active.head()

In [ ]:
df_inactive.head()

In [ ]:
col = ["id","name","symbol","slug","is_active","first_historical_data","last_historical_data"]
df = pd.concat([df_active[col],df_inactive[col]]) \
       .dropna(thresh=6) \
       .reset_index(drop=True) # first_historical_data가 NaN인 친구들은 drop 

In [ ]:
df.to_csv("coin_list_coinmarketcap.csv")

# OHLCV Download

In [ ]:
coin_list = pd.read_csv("coin_list_coinmarketcap.csv", index_col=0)
coin_list.head()

In [ ]:
#coin_list.iloc[-100:-50]

In [ ]:
query_id = str(list(coin_list.id.values.tolist()))[1:-1].replace(" ","")
query_id

In [ ]:
numbers_list = list(map(int, query_id.split(",")))

# Initialize an empty list to store the groups
groups = []

# Iterate over the list of numbers and split them into groups of five
for i in range(0, len(numbers_list), 5):
    group = numbers_list[i:i+5]
    groups.append(group)

# Convert each group into a string and join them with commas
groups_strings = [",".join(map(str, group)) for group in groups]

In [ ]:
def get_coin_ohlcv(group_string):
    url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/ohlcv/historical'

    headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': 'e0070978-f2c5-4a31-b9b8-7705d84077bd',
    }

    params = {
        "id" : group_string,
        "time_period" : "daily", # hourly 가능...
        "time_start" : "2022-04-15",   
        "time_end":"2022-04-16",
        "count":10000
        }

    res = requests.get(url=url, headers=headers, params=params)
    return res

        "time_start" : "2018-04-17",
        "time_end":"2023-04-16",

In [ ]:
len(groups_strings)
# 500개씩 나눠서 보내자

In [ ]:
return_dict = {}

for string in tqdm(groups_strings[:500]):
    s = string.split(",")[0]
    time.sleep(0.4)
    return_dict["g"+str(s)] = get_coin_ohlcv(string)

In [ ]:
final_list = []
error_key = {}

for key, values in tqdm(return_dict.items()):
    try:
        data = values.json()["data"]
        final_dict = {}
        
        for id, json in data.items():
            df = pd.DataFrame(json["quotes"])
            df_time = df.iloc[:,0:4]

            quote = df.quote
            open = quote.apply(lambda x: x["USD"]["open"])
            open.name = "open"
            close = quote.apply(lambda x: x["USD"]["close"])
            close.name = "close"
            low = quote.apply(lambda x: x["USD"]["low"])
            low.name = "low"
            high = quote.apply(lambda x: x["USD"]["high"])
            high.name = "high"
            vol = quote.apply(lambda x: x["USD"]["volume"])
            vol.name = "vol"
            mktcap = quote.apply(lambda x: x["USD"]["market_cap"])
            mktcap.name = "mktcap"

            concat_df = pd.concat([df_time, open,close, low, high,vol,mktcap], axis=1)
            final_dict[id] = concat_df
    except:
        error_key[id] = json
        print("Error Raised!!")
        
    final_list.append(final_dict)

In [ ]:
len(final_list)

In [ ]:
# Open, close, low, high, vol, mktcap, time 각각 따로 저장하자
df_close = pd.DataFrame()
df_low = pd.DataFrame()
df_high = pd.DataFrame()
df_vol = pd.DataFrame()
df_mktcap = pd.DataFrame()
df_open = pd.DataFrame()

df_time_high = pd.DataFrame()
df_time_low = pd.DataFrame()

for data_dict in tqdm(final_list):
    for coin_key, df in data_dict.items():
        df.index = pd.to_datetime(df.time_open).dt.strftime('%Y-%m-%d')
        
        df_open[coin_key] = df.open
        df_close[coin_key] = df.close
        df_low[coin_key] = df.low
        df_high[coin_key] = df.high
        df_vol[coin_key] = df.vol
        df_mktcap[coin_key] = df.mktcap
        
        df_time_high[coin_key] = df.time_high
        df_time_low[coin_key] = df.time_low

In [ ]:
(df_open.columns == df_mktcap.columns).mean()#[["1"]].plot(figsize=(24,12));

In [ ]:
# Save
#df_close.to_pickle("df_close3.pickle")
#df_low.to_pickle("df_low3.pickle")
#df_high.to_pickle("df_high3.pickle")
#df_vol.to_pickle("df_vol3.pickle")
#df_mktcap.to_pickle("df_mktcap3.pickle")
#df_open.to_pickle("df_open3.pickle")
#
#df_time_high.to_pickle("df_time_high3.pickle")
#df_time_low.to_pickle("df_time_low3.pickle")

## Data Concat

In [ ]:
d0,d1,d2,d3 = {},{},{},{}
i = 0

for dict in [d0,d1,d2,d3]:
    for path in os.listdir(f"coinmarketcap_data/{str(i)}"):
        read = pd.read_pickle(f"coinmarketcap_data/{str(i)}/{path}")
        read.index = pd.to_datetime(read.index)
        dict[path[:-7]] = read
    i += 1

In [ ]:
print(d0.keys())
print(d1.keys())
print(d2.keys())
print(d3.keys())